In [1]:
import pandas as pd

In [2]:
path = "results/"
filename = "batch_profiling_results_red_cuda_20240906_123954_heavymodels_fp16.csv"
df = pd.read_csv(path+filename)

In [5]:
df.groupby(['model_alias', 'batch_size']).mean().reset_index()

,model_alias,batch_size,processing_time,throughput
0,gemma-7b,1,0.933442,1.072
1,gemma-7b,2,0.949676,2.106
2,gemma-7b,4,0.930215,4.302
3,gemma-7b,8,1.225456,6.568
4,gemma-7b,16,1.414788,11.350
5,gemma-7b,32,2.214126,14.676
6,gemma-7b,64,3.542751,18.062
7,gemma-7b,128,6.785152,18.896
8,gemma-7b,256,NaN,NaN
9,granite-7b,1,0.998737,1.046
